In [1]:
#data generation for down pendulum
import numpy as np
import scipy.integrate
import random

random.seed(32)  # Python random seed
np.random.seed(32)  # NumPy random seed

print("Generating data")

# Simulation variables
seq_length = 5001 # Number of time steps in each sequence
num_records = 50 # Number of sequences to generate
dim_y = 1  # No. of observables
dim_x= 2 # Number of states
dim_u=1 #shape of control input
dt=0.01
end_t=seq_length*dt
t_span= np.arange(0,end_t,dt)
num_timesteps=np.shape(t_span)[0]
U_step = np.where(t_span >= 1.0, 1.0, 0.0)

# Arrays to store results
U_data_array = np.zeros((num_records, seq_length, dim_u))
X_data_array = np.empty((num_records, seq_length, dim_x))
Y_data_array = np.empty((num_records, seq_length, dim_y))

# Dynamics function with control 
def f(t, x, u):
    x1, x2 = x
    g, l, b, m = 9.8, 1.0, 0.9, 2.0
    dx1 = x2
    dx2 = -(g/l)*np.sin(x1) - (b/m)*x2 + (1.0/m)*u  # external torque
    return np.array([dx1, dx2])

# Euler integrator
def step_forward(t,x,u,dt):
    return x + dt * f(t,x,u)
    
# Parameters
l=1
g=9.8 
m=2
b=0.9

P=0.01*np.identity(dim_x) #covariance of initial condition
mu_pn = np.zeros(dim_x) #mean of process noise
Q=0.01*np.identity(dim_x) #process noise covariance
mu_mn=np.zeros(dim_y) #mean of measurement noise
R=[0.01] #measurement noise covariance
G=np.identity(dim_x) #process noise coefficient matrix
B = np.array([[0.0], [1.0 / m]]) # Control matrix: affects angular acceleration # control affects second state (torque)
H=np.array(([[1.0],[0.0]])) #measurment matrix

mu_x0=np.zeros(dim_x)

for i in range(0, num_records, 1):
    x=np.random.multivariate_normal(mu_x0,P)

    # Control signal with small noise
    u_sequence = U_step + 0.1 * np.random.randn(seq_length)
    U_data_array[i, :, 0] = u_sequence
    X_data_array[i, 0, :] = x
    Y_data_array[i, 0, :] = np.matmul(x,H) + np.random.normal(mu_mn, R)

    for j in range(1, seq_length):
        u = u_sequence[j]
        x = step_forward(t_span[j-1], x, u, dt)
        w = np.random.multivariate_normal(mu_pn, Q)
        x = x + G @ w
        X_data_array[i, j, :] = x

        v = np.random.normal(mu_mn, R)
        Y_data_array[i, j, :] = np.matmul(x,H) + v
        
np.savez("downpendseq_data_fixed_mean.npz", X_data=X_data_array[:,1:,:], Y_data=Y_data_array[:,1:,:], U_data=U_data_array[:,1:,:])


Generating data


In [2]:
#for different initial condition

# Arrays to store results
U_data_array_diff_ic = np.zeros((num_records, seq_length, dim_u))
X_data_array_diff_ic = np.empty((num_records, seq_length, dim_x))
Y_data_array_diff_ic = np.empty((num_records, seq_length, dim_y))

# Simulation loop
for i in range(num_records):
    mu_x0 = np.random.uniform(-10, 10, size=dim_x)
    x = np.random.multivariate_normal(mu_x0, P)

    # Control signal with small noise
    u_sequence = U_step + 0.1 * np.random.randn(seq_length)
    U_data_array_diff_ic[i, :, 0] = u_sequence

    X_data_array_diff_ic[i, 0, :] = x
    Y_data_array_diff_ic[i, 0, :] = np.matmul(x,H) + np.random.normal(mu_mn, R)

    for j in range(1, seq_length):
        u = u_sequence[j]
        x = step_forward(t_span[j-1], x, u, dt)
        w = np.random.multivariate_normal(mu_pn, Q)
        x = x + G @ w
        X_data_array_diff_ic[i, j, :] = x

        v = np.random.normal(mu_mn, R)
        Y_data_array_diff_ic[i, j, :] = np.matmul(x,H) + v
        
np.savez("downpendseq_data_diff_ic_fixed_mean.npz", X_data=X_data_array_diff_ic[:,1:,:], Y_data=Y_data_array_diff_ic[:,1:,:], U_data=U_data_array_diff_ic[:,1:,:])
